In [2]:
import nltk
from nltk.corpus import brown
from nltk.corpus import stopwords
#from baiye import BayesianFilter
#bf = BayesianFilter()
import math, sys
from konlpy.tag import Twitter
import re
import random

In [3]:
class BayesianFilter:
    def __init__(self):
        self.words = set() # 출현한 단어 기록
        self.word_dict = {} # 카테고리마다의 출현 횟수 기록
        self.category_dict = {} # 카테고리 출현 횟수 기록
        self.cfd = _
        
    # 형태소 분석하기 --- (※1)
    def split(self, text):
        return text.split()
        results = []
        twitter = Twitter()
        # 단어의 기본형 사용
        malist = twitter.pos(text, norm=True, stem=True)
        for word in malist:
            # 어미/조사/구두점 등은 대상에서 제외 
            if not word[1] in ["Josa", "Eomi", "Punctuation"]:
                results.append(word[0])
        return results
    
    def fit_cfd(self, cfd):
        
        for category in cfd.conditions():
            
            for word in cfd[category]:
                if not category in self.category_dict:
                    self.category_dict[category] = 0
                self.category_dict[category] += 1
                
                if not category in self.word_dict:
                    self.word_dict[category] = {}
                if not word in self.word_dict[category]:
                    self.word_dict[category][word] = 0
                self.word_dict[category][word] = cfd[category][word]
                self.words.add(word)

    def fit_list(self, data):
        
        training_data = []
        
        for c, w in data:
            if not c in self.category_dict:
                   self.category_dict[c] = 0
                    
            self.category_dict[c] += 1
            self.words.add(w)
            
            training_data += {(c,w)}
            
        self.cfd = nltk.ConditionalFreqDist(training_data)
        
    # 단어 리스트에 점수 매기기--- (※4)
    def score(self, words, category):
        s_category = sum(self.category_dict.values())         
        score = math.log( self.category_dict[category] / s_category )
        for word in words:
            #score += math.log(self.word_prob(word, category))
            score += math.log( (self.cfd[category][word]+1) / (self.category_dict[category] + len(self.words)) )
        return score
    
    # 예측하기 --- (※5)
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize
        #print(text)
        words = nltk.pos_tag(text)
        score_list = []
        for category in self.category_dict.keys():
            score = self.score(words, category)
            score_list.append((category, score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list

    # 카테고리 내부의 단어 출현 비율 계산 --- (※6)
    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1 # ---(※6a)
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n / d

def content_prep(text, stopwords):
    #stopwords = nltk.corpus.stopwords.words('english')
    content = []
    wnl = nltk.WordNetLemmatizer()
    
    for c,w in text:
        #word = wnl.lemmatize(w)
        word = w
        if word.lower() not in stopwords and re.search('[A-Za-z0-9]$', word):
        #if word.lower() not in stopwords:
            content+={(c, word)}
    return content

In [4]:
genre_word = [(genre,sent) 
              for genre in ['romance','science_fiction','news']
              for sent in  brown.sents(categories=genre)]
              #for content_fraction(word,stopwords) in brown.words(categories=genre)]

In [5]:
random.shuffle(genre_word)

In [6]:
train_set, test_set = genre_word[:int(len(genre_word)*0.9)],genre_word[int(len(genre_word)*0.9):]

In [7]:
train_data = []
for c, s in train_set:
    words = nltk.pos_tag(s)
    for w in words:
        train_data += {(c,w)}

In [8]:
train_data

[('news', ('Pelham', 'NNP')),
 ('news', ('pointed', 'VBD')),
 ('news', ('out', 'RP')),
 ('news', ('that', 'IN')),
 ('news', ('Georgia', 'NNP')),
 ('news', ('voters', 'NNS')),
 ('news', ('last', 'JJ')),
 ('news', ('November', 'NNP')),
 ('news', ('rejected', 'VBD')),
 ('news', ('a', 'DT')),
 ('news', ('constitutional', 'JJ')),
 ('news', ('amendment', 'NN')),
 ('news', ('to', 'TO')),
 ('news', ('allow', 'VB')),
 ('news', ('legislators', 'NNS')),
 ('news', ('to', 'TO')),
 ('news', ('vote', 'VB')),
 ('news', ('on', 'IN')),
 ('news', ('pay', 'NN')),
 ('news', ('raises', 'NNS')),
 ('news', ('for', 'IN')),
 ('news', ('future', 'JJ')),
 ('news', ('Legislature', 'NNP')),
 ('news', ('sessions', 'NNS')),
 ('news', ('.', '.')),
 ('science_fiction', ('``', '``')),
 ('science_fiction', ("I'll", 'NNP')),
 ('science_fiction', ('carry', 'VBP')),
 ('science_fiction', ('the', 'DT')),
 ('science_fiction', ('tray', 'NN')),
 ('science_fiction', ("''", "''")),
 ('science_fiction', ('.', '.')),
 ('news', ('How

In [10]:
bf = BayesianFilter()
bf.fit_list(train_data)

print(test_set)

[('news', ['Dr.', 'and', 'Mrs.', 'N.', 'Volney', 'Ludwick', 'have', 'had', 'as', 'guests', 'Mr.', 'and', 'Mrs.', 'John', 'J.', 'Evans', ',', 'Jr.', ',', 'of', '``', 'Kimbolton', 'House', "''", ',', 'Rockhall', ',', 'Md.', '.']), ('news', ['``', 'Are', 'you', 'kidding', "''", '?', '?']), ('romance', ['It', 'swayed', 'with', 'the', 'wicker', 'swings', 'and', 'screeched', 'with', 'the', 'rusted', 'hinges', 'of', 'screen', 'doors', '.']), ('romance', ['He', 'described', 'the', 'ham', 'decorated', 'with', 'pineapple', 'and', 'cherries', ',', 'cooling', 'on', 'the', 'porch', '.']), ('news', ['Alla', 'Sizova', ',', 'who', 'seems', 'to', 'have', 'made', 'a', 'special', 'hit', 'in', 'the', 'East', ',', 'was', 'delightful', 'as', 'the', 'lady', 'Bluebird', 'and', 'her', 'partner', ',', 'Yuri', 'Soloviev', ',', 'was', 'wonderfully', 'virile', ',', 'acrobatic', ',', 'and', 'poetic', 'all', 'at', 'the', 'same', 'time', ',', 'in', 'a', 'tradition', 'not', 'unlike', 'that', 'of', 'Nijinsky', '.']), (

In [318]:
# 예측
accuracy = 0.0
t_count = 0

for c,w in test_set:
    pre, scorelist = bf.predict(w)
    if(pre == c):
        t_count += 1
    #print('s word',w)
    #print('s cate',c)
    #print("결과 =", pre)
    #print("결과 =", scorelist)
    #print()
print(round(t_count/len(test_set),3)*100,'%')

82.89999999999999 %


In [315]:
s = ('love', 'VB')
bf.predict(s)

('romance',
 [('science_fiction', -22.37578237518578),
  ('romance', -20.620935101516558),
  ('news', -23.874933625498144)])